## Advanced Login

By default, the client will create a redirect server and launch the user's default browser to show the login page. Neither of these are however required. This is useful for deployed applications that need to be initialized from the command line (also known as "headless authentication").

Start by initializing the client as usual:

In [1]:
from saxo_apy import SaxoOpenAPIClient

client = SaxoOpenAPIClient()

Next, use the `.login()` method with the below arguments to disable the browser and redirect server.

You will manually have to open the login page at the provided link, and copy the URL location you land on into the terminal after you enter your credentials.

Note: because no server is listening for the redirect, the browser will show a "This site can't be reached" or "Unable to connect" error. This is however not a problem, as the required data (the authorization code) is available in the URL's query parameters. Copy the entire URL into your command line when prompted to connect the client.

In [ ]:
client.login(launch_browser=False, catch_redirect=False, start_async_refresh=True)

The command line will look something like this:

```
🌐 navigate to the following web page to log in: https://sim.logonvalidation.net/authorize?.....
📎 paste redirect location (url): < REDIRECT LOCATION GOES HERE >
✅ authorization succeeded - connected to 🛠 SIM environment with 🔧 write / 📈 trade permissions (session ID ea1df9dfd8464915947ba529bd2f02ac)
```

The client is now connected in the same way as using the browser/redirect server:

In [3]:
client.logged_in

True

In [4]:
# if start_async_refresh was set when logging in, the asyncio tasks list will confirm this (only for Jupyter Notebooks)
import asyncio

[task.get_name() for task in asyncio.all_tasks() if task.get_name() == "async_refresh"]

['async_refresh']

## Authenticating a Deployed Application

Let's assume you have a cloud server running at `my-dev-box.my-domain.net`. We want to deploy a OpenAPI client on this server and authenticate it with Saxo SSO.

We can use the above procedure to authenticate the client from the command line (through SSH for example).

However, we can also use the client to seamlessly authenticate while still using a browser and callback server. We first need to ensure we add a new redirect URL to the application on the Developer Portal:

`http://my-dev-box.my-domain.net:12321/redirect`

Saxo SSO will redirect the user to this location after login credentials are validated. Make sure your cloud instance firewall allows for inbound connections on the chosen port (in this case `12321`), otherwise your server will never be able to catch the callback.

Next, we initialize the client as usual:

In [1]:
from saxo_apy import SaxoOpenAPIClient

client = SaxoOpenAPIClient()

The client provides a convenience function to show the available redirect URLs:

In [ ]:
[url.host for url in client.available_redirect_urls]
# ['localhost', 'my-dev-box.my-domain.net']

Next, we select the **second** URL for our login flow:

In [3]:
client.login(redirect_url=client.available_redirect_urls[1])

🌐 opening login page in browser - waiting for user to authenticate... 🔑
📞 received callback from Saxo SSO
✅ authorization succeeded - connected to SIM environment with WRITE / TRADE permissions (session ID 0499351bebc345df80a0070aab04d151)


Enter your credentials and confirm. Saxo SSO now forwards you to the cloud instance, which should be able to catch the callback and authorize your client without you needing to copy URLs back and forth!

## Authenticating a Deployed Application Behind a Reverse Proxy with HTTPS Enabled

In the above example the redirect to the server listening on port 12321 uses the insecure HTTP protocol. The user's browser can therefore not verify that the machine it's connecting to _is_ actually `my-dev-box.my-domain.net`. This poses a security risk because the `auth_code` param that is included in the redirect could potentially fall into the wrong hands.

To ensure security, we need to redirect to: `https://my-dev-box.my-domain.net` instead (notice HTTP**S** instead of HTTP).

This requires the following steps:
- The devbox needs some reverse proxy to offload SSL. In this example I am using Nginx with Let's Encrypt/Certbot on an Ubuntu machine (for an excellent tutorial on how to set this up, see DigitalOcean's setup [guide 1](https://www.digitalocean.com/community/tutorials/how-to-install-nginx-on-ubuntu-20-04) and [guide 2](https://www.digitalocean.com/community/tutorials/how-to-secure-nginx-with-let-s-encrypt-on-ubuntu-20-04)).
- The app config needs to be updated with a new redirect url: `https://my-dev-box.my-domain.net:443/redirect` (notice HTTP**S** and port=443).
- We need to configure the `saxo-apy` redirect server to listen on a different port, because port 443 will be used by Nginx to receive incoming traffic (which is then proxy'd to the redirect server).

The Nginx configuration for your domain should be extended with some `proxy_pass` redirect such as:

```
    location /redirect {
        proxy_pass http://127.0.0.1:8080/redirect;
    }
```

This will receive incoming (SSL-secured) traffic on the route to `/redirect` and forward the request to localhost port 8080. This will be the location where our redirect server is catching the request to handle the next steps of the authorization process.

Now, we can connect our application to Saxo securely over HTTPS. Make sure to update your local copy of the application configuration (which should have the new redirect URL specified above).

In [1]:
from saxo_apy import SaxoOpenAPIClient

client = SaxoOpenAPIClient()

In [2]:
client.login(
    redirect_url=client.available_redirect_urls[2],  # in this case the 3rd url in my config is the HTTPS redirect
    redirect_port=8080,  # this is the same port as in the above Nginx config
)

🌐 opening login page in browser - waiting for user to authenticate... 🔑
📞 received callback from Saxo SSO
✅ authorization succeeded - connected to SIM environment with WRITE / TRADE permissions (session ID 75950811b4d7441f9977cbe01e16ec36)


Notice the SSL symbol in your browser when connecting to your devbox (typically a closed lock icon). You are now connected to Saxo OpenAPI with a secured redirect flow!